In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create Vertex AI Matching Engine index - Streaming w/ PSC
![ ](https://www.google-analytics.com/collect?v=2&tid=G-L6X3ECH596&cid=1&en=page_view&sid=1&dt=sdk_matching_engine_for_indexing.ipynb&dl=notebooks%2Fofficial%2Fmatching_engine%2Fsdk_matching_engine_for_indexing.ipynb)
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
      <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/matching_engine/sdk_matching_engine_for_indexing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This example demonstrates how to use the Vertex AI ANN Service. It is a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

Learn more about [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview).

### Objective

In this notebook, you learn how to create Approximate Nearest Neighbor (ANN) Index, query against indexes, and validate the performance of the index. 

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Matching Engine`

The steps performed include:

* Create ANN Index and Brute Force Index
* Create an IndexEndpoint with Private Service Connect (PSC)
* Deploy ANN Index and Brute Force Index
* Perform online query
* Compute recall


### Dataset

The dataset used for this tutorial is the [GloVe dataset](https://nlp.stanford.edu/projects/glove/).

"GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space."


## Create Workbench Instance with Conda Env as Kernel

Create a Vertex Workbench Instance [here](https://console.cloud.google.com/vertex-ai/workbench/instances/create)

### Create a new conda environment to use as a jupyter kernel

Run the following commands in the terminal of your Vertex Workbench Instance

1. Create a new conda environment with Python 3.10 and ipykernel
```bash
CONDA_ENV_NAME="vme"
conda create -n $CONDA_ENV_NAME python=3.10 ipykernel
```
2. Change the display name of the jupyter kernel to Python 3 (env_name)
```bash
mv /opt/conda/envs/$CONDA_ENV_NAME/share/jupyter/kernels/python3/kernel.json /tmp/temp.json && jq -r '.display_name |= "Python 3 ('$CONDA_ENV_NAME')"' /tmp/temp.json > /opt/conda/envs/$CONDA_ENV_NAME/share/jupyter/kernels/python3/kernel.json && rm /tmp/temp.json
```
3. Deactivate conda environment
```bash
conda deactivate
```

### Select your new jupyter kernel for this jupyter notebook

## Installation

Install the latest version of Cloud Storage, BigQuery and Vertex AI SDKs for Python.

Install the `h5py` to prepare sample dataset, and the `grpcio-tools` for querying against the index. 

In [4]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        grpcio-tools \
                        h5py

  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/8f/3f/979e65dc13c11b4c53241d105d41d4b937b131e56b68397cc279f372be96/google_cloud_aiplatform-1.28.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-storage from https://files.pythonhosted.org/packages/88/14/c9d4faae7ea4bff4405152cbc762b61100aa6949273b4eb3203d23308670/google_cloud_storage-2.10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for grpcio-tools from https://files.pythonhosted.org/packages/ef/86/499bfeb0a1e27c5f5b0204964112f345a1b8bb262b05992b52319e381bb6/grpcio_tools-1.56.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/0d/7a/e55589e4093cca1934db5e99644c1c2424a9b3aac104b7f6176605a5eeb7/h5py-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for google-a

Restart kernel

## Before you begin
#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [1]:
PROJECT_ID = "wmt-7fbls2a91f025anb93e025b02g"  # @param {type:"string"}

# Set the project id
! echo Y | gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [7]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

## Make sure the following cells are run from inside the VPC network that you created in the previous step.

* **WARNING:** The MatchingIndexEndpoint.match method (to create online queries against your deployed index) has to be executed in a Vertex AI Workbench notebook instance that is created with the following requirements:
  * **In the same region as where your ANN service is deployed** (for example, if you set `REGION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  * If you run it in the colab or a Vertex AI Workbench notebook instance in a different VPC network or region, "Create Online Queries" section will fail.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = "gs://wmt-aug23-vertexgenai-workshop-data/matching-engine"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [12]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://embeddings-for-kroger/...


## Prepare the data

The GloVe dataset consists of a set of pre-trained embeddings. The embeddings are split into a "train" split, and a "test" split.
We will create a vector search index from the "train" split, and use the embedding vectors in the "test" split as query vectors to test the vector search index.

**Note:** While the data split uses the term "train", these are pre-trained embeddings and therefore are ready to be indexed for search. The terms "train" and "test" split are used just to be consistent with machine learning terminology.

Download the GloVe dataset.


In [4]:
! gsutil -m cp gs://wmt-aug23-vertexgenai-workshop-data/matching-engine/data/glove-100-angular.hdf5 .

Copying gs://wmt-aug23-vertexgenai-workshop-data/matching-engine/data/glove-100-angular.hdf5...
- [1/1 files][462.9 MiB/462.9 MiB] 100% Done                                    
Operation completed over 1 objects/462.9 MiB.                                    


Read the data into memory.


In [4]:
import h5py

# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]

In [5]:
train[0]

array([-0.11333  ,  0.48402  ,  0.090771 , -0.22439  ,  0.034206 ,
       -0.55831  ,  0.041849 , -0.53573  ,  0.18809  , -0.58722  ,
        0.015313 , -0.014555 ,  0.80842  , -0.038519 ,  0.75348  ,
        0.70502  , -0.17863  ,  0.3222   ,  0.67575  ,  0.67198  ,
        0.26044  ,  0.4187   , -0.34122  ,  0.2286   , -0.53529  ,
        1.2582   , -0.091543 ,  0.19716  , -0.037454 , -0.3336   ,
        0.31399  ,  0.36488  ,  0.71263  ,  0.1307   , -0.24654  ,
       -0.52445  , -0.036091 ,  0.55068  ,  0.10017  ,  0.48095  ,
        0.71104  , -0.053462 ,  0.22325  ,  0.30917  , -0.39926  ,
        0.036634 , -0.35431  , -0.42795  ,  0.46444  ,  0.25586  ,
        0.68257  , -0.20821  ,  0.38433  ,  0.055773 , -0.2539   ,
       -0.20804  ,  0.52522  , -0.11399  , -0.3253   , -0.44104  ,
        0.17528  ,  0.62255  ,  0.50237  , -0.7607   , -0.071786 ,
        0.0080131, -0.13286  ,  0.50097  ,  0.18824  , -0.54722  ,
       -0.42664  ,  0.4292   ,  0.14877  , -0.0072514, -0.1648

#### Save the train split in JSONL format.

The data must be formatted in JSONL format, which means each embedding dictionary is written as a JSON string on its own line.

Additionally, to demonstrate the filtering functionality, the `restricts` key is set such that each embedding has a different `class`, `even` or `odd`. These are used during the later matching step to filter for results.
See additional information of filtering here: https://cloud.google.com/vertex-ai/docs/matching-engine/filtering

In [7]:
import json

with open("glove100.json", "w") as f:
    embeddings_formatted = [
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(value) for value in embedding],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
        for index, embedding in enumerate(train)
    ]
    f.writelines(embeddings_formatted)

Upload the training data to GCS.

In [6]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/matching_engine_demo/initial/"
! gsutil -m cp glove100.json {EMBEDDINGS_INITIAL_URI}

## Create Indexes


### Create ANN Index (for Production Usage)

In [7]:
DIMENSIONS = 100
DISPLAY_NAME = "stream_glove_100_1"
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

Create the ANN index configuration:

To learn more about configuring the index, see [Input data format and structure](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure).


In [8]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
from google.protobuf import struct_pb2

In [9]:
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"
ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [46]:
index_client = aiplatform_v1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [47]:
treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=500),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=7),
    }
)

algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
        "shardSize": struct_pb2.Value(string_value="SHARD_SIZE_SMALL")
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=EMBEDDINGS_INITIAL_URI),
    }
)

ann_index = {
    "display_name": DISPLAY_NAME,
    "description": "Glove 100 ANN index - stream",
    "metadata": struct_pb2.Value(struct_value=metadata),
    "index_update_method": aiplatform_v1.Index.IndexUpdateMethod.STREAM_UPDATE,
}
     

In [48]:
ann_index = index_client.create_index(parent=PARENT, index=ann_index)

In [16]:
ann_index.result()


name: "projects/896267025569/locations/us-central1/indexes/7485114522085097472"

In [17]:
INDEX_RESOURCE_NAME = ann_index.result().name
INDEX_RESOURCE_NAME

'projects/896267025569/locations/us-central1/indexes/7485114522085097472'

In [10]:
INDEX_RESOURCE_NAME="3053572488752529408"

Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [11]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method is not fast or efficient. Hence brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors, so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an apples to apples comparison, the `distanceMeasureType` and `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [54]:
algorithmConfig = struct_pb2.Struct(
    fields={"bruteForceConfig": struct_pb2.Value(struct_value={})}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=EMBEDDINGS_INITIAL_URI),
    }
)

ann_index = {
    "display_name": DISPLAY_NAME_BRUTE_FORCE,
    "description": "Glove 100 index (brute force) - stream",
    "metadata": struct_pb2.Value(struct_value=metadata),
    "index_update_method": aiplatform_v1.Index.IndexUpdateMethod.STREAM_UPDATE,
}
     

In [55]:
ann_index = index_client.create_index(parent=PARENT, index=ann_index)

In [ ]:
ann_index.result()


In [50]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = ann_index.result().name
INDEX_BRUTE_FORCE_RESOURCE_NAME

AttributeError: 'dict' object has no attribute 'result'

In [51]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = "3778652028759179264"

In [52]:
brute_force_index = aiplatform.MatchingEngineIndex(
    index_name=INDEX_BRUTE_FORCE_RESOURCE_NAME
)

## This takes 20-30 minutes - here's some reading on what it is doing
#### Note on the advantages of the algorithm

[link](https://arxiv.org/pdf/1908.10396.pdf)

```However, it is easy to see that not all pairs of (x, q) are equally important. The approximation error on the pairs which have a high inner product is far more important since they are likely to be among the top ranked pairs and can greatly affect the search result, while for the pairs whose inner product is low the approximation error matters much less. In other words, for a given datapoint x, we should quantize it with a bigger focus on its error with those queries which have high inner product with x. See Figure 1 for the illustration.```


![](img/algo.png)

## Other quick notes on ME while we wait for deployment

[Blog Link on the technology](https://cloud.google.com/blog/topics/developers-practitioners/find-anything-blazingly-fast-googles-vector-search-technology)

Instead of comparing vectors one by one, you could use the approximate nearest neighbor (ANN) approach to improve search times. Many ANN algorithms use vector quantization (VQ), in which you split the vector space into multiple groups, define "codewords" to represent each group, and search only for those codewords. This VQ technique dramatically enhances query speeds and is the essential part of many ANN algorithms, just like indexing is the essential part of relational databases and full-text search engines.

![](img/vectorQuant.gif)


As you may be able to conclude from the diagram above, as the number of groups in the space increases the speed of the search decreases and the accuracy increases.  Managing this trade-off — getting higher accuracy at shorter latency — has been a key challenge with ANN algorithms. 

Last year, Google Research announced ScaNN, a new solution that provides state-of-the-art results for this challenge. With ScaNN, they introduced a new VQ algorithm called anisotropic vector quantization:

![](img/Loss_Types.max-1000x1000.png)

Anisotropic vector quantization uses a new loss function to train a model for VQ for an optimal grouping to capture farther data points (i.e. higher inner product) in a single group. With this idea, the new algorithm gives you higher accuracy at lower latency, as you can see in the benchmark result below (the violet line): 

![](img/speedvsaccuracy.max-1600x1600.png)

[You can see 20% increase in performance vs. standard SOLR-type algos , 15% ish for FAISS](https://ann-benchmarks.com/)


### Let's also quickly break down the parameters and suggeted settings

```python
  "contentsDeltaUri": "gs://BUCKET_NAME/path",
  "config": {
    "dimensions": 100,
    "approximateNeighborsCount": 150,
    "distanceMeasureType": "DOT_PRODUCT_DISTANCE",
    "shardSize": "SHARD_SIZE_MEDIUM",
    "algorithm_config": {
      "treeAhConfig": {
        "leafNodeEmbeddingCount": 5000,
        "leafNodesToSearchPercent": 3
      }
    }
  }
```

- `dimensions` size of the embeddings stored in the index
- `approximateNeighborsCount` set this to increase approximate results for a search. Latency increases proprotinately, but recall does as well
- `shardSize` defaults to medium, but general rule of thumb is to try to go as large as budget allows - going to a large shard can often help performance
- `distanceMeasureType` can be squared l2, l1, cosine, dot product, unit l2

#### config details (important still!)
- `leafNodeEmbeddingCount` - number of embeddings in each cluster/leaf
- `leafNodesToSearchPercent` - the percentage of leaf/clusters searched for any given query - this number is an int but treated as a percent

## Stream Update Indexes

#### Note from the PM on Compaction

`Periodically, your index is rebuilt to account for all new updates since your last rebuild. This rebuild, or "compaction", improves query performance and reliability. Compactions occur for both Streaming Updates and Batch Updates.`

```
Streaming Update: Occurs when the uncompacted data size is > 1 GB or the oldest uncompacted data is at least three days old. You are billed for the cost of rebuilding the index at the same rate of a batch update, in addition to the Streaming Update costs.
Batch Update: Occurs when the incremental dataset size is > 20% of the base dataset size.
```

### Note on autoscaling too

You can autoscale when creating in index in gcloud (not the SDK). Example is [here](https://cloud.google.com/vertex-ai/docs/matching-engine/deploy-index-public#autoscaling), however mutation of an existing index is recommended since we will create with the SDK



Notes from the PM on bursting to large scale (help with big retailer): 

```
We support autoscaling. Just set max-replica-count to a large enough number, ME will respond the the peak load by increasing the number of replicas (gradually). Beneath the surface, autoscaling is handled by GKE (HPA) and scale up happens when the CPU load of existing pods exceeds 50%. That being said, the autocaling can be slow - new pods need 30 - 60 minutes to be ready and the scaling up is like a staircase. Scale down is usually much faster. When the pods are overloaded, the throttle/reject new requests and return RESOURCE_EXHAUSTED errors. If the customer knows when the peak comes, we recommend manual scaling instead - manually update the deployed index to set the min-replica-count to a high enough value
```

[Link to mutating resource config on an index](https://cloud.google.com/vertex-ai/docs/matching-engine/deploy-index-public#autoscaling)


Best resources for QPS

n2d-standard-32 is the most powerful machine type we have and it's most cost effective for heavy/batch load.
Each n2d-standard-32  can handle 1k - 10k QPS depends on lots of factors. 


In [23]:
# Test query
query = [
    -0.11333,
    0.48402,
    0.090771,
    -0.22439,
    0.034206,
    -0.55831,
    0.041849,
    -0.53573,
    0.18809,
    -0.58722,
    0.015313,
    -0.014555,
    0.80842,
    -0.038519,
    0.75348,
    0.70502,
    -0.17863,
    0.3222,
    0.67575,
    0.67198,
    0.26044,
    0.4187,
    -0.34122,
    0.2286,
    -0.53529,
    1.2582,
    -0.091543,
    0.19716,
    -0.037454,
    -0.3336,
    0.31399,
    0.36488,
    0.71263,
    0.1307,
    -0.24654,
    -0.52445,
    -0.036091,
    0.55068,
    0.10017,
    0.48095,
    0.71104,
    -0.053462,
    0.22325,
    0.30917,
    -0.39926,
    0.036634,
    -0.35431,
    -0.42795,
    0.46444,
    0.25586,
    0.68257,
    -0.20821,
    0.38433,
    0.055773,
    -0.2539,
    -0.20804,
    0.52522,
    -0.11399,
    -0.3253,
    -0.44104,
    0.17528,
    0.62255,
    0.50237,
    -0.7607,
    -0.071786,
    0.0080131,
    -0.13286,
    0.50097,
    0.18824,
    -0.54722,
    -0.42664,
    0.4292,
    0.14877,
    -0.0072514,
    -0.16484,
    -0.059798,
    0.9895,
    -0.61738,
    0.054169,
    0.48424,
    -0.35084,
    -0.27053,
    0.37829,
    0.11503,
    -0.39613,
    0.24266,
    0.39147,
    -0.075256,
    0.65093,
    -0.20822,
    -0.17456,
    0.53571,
    -0.16537,
    0.13582,
    -0.56016,
    0.016964,
    0.1277,
    0.94071,
    -0.22608,
    -0.021106,
]
    

### Insert Datapoints

In [24]:
insert_datapoints_payload = aiplatform_v1.IndexDatapoint(
    datapoint_id="101",
    feature_vector=query,
    restricts=[{"namespace": "class", "allow_list": ["odd"]}],
)

upsert_request = aiplatform_v1.UpsertDatapointsRequest(
    index=tree_ah_index.resource_name, datapoints=[insert_datapoints_payload]
)

index_client.upsert_datapoints(request=upsert_request)

### Edit Datapoints

Changing restricts namespace value from odd to even

In [25]:
insert_datapoints_payload = aiplatform_v1.IndexDatapoint(
    datapoint_id="101",
    feature_vector=query,
    restricts=[{"namespace": "class", "allow_list": ["even"]}],
)

upsert_request = aiplatform_v1.UpsertDatapointsRequest(
    index=tree_ah_index.resource_name, datapoints=[insert_datapoints_payload]
)

index_client.upsert_datapoints(request=upsert_request)

### Remove Datapoints

In [29]:
# Remove the datapoint with id '101' from the index
remove_request = aiplatform_v1.RemoveDatapointsRequest(
    index=tree_ah_index.resource_name, datapoint_ids=["101"]
)

index_client.remove_datapoints(request=remove_request)

## Create an IndexEndpoint with Private Service Connect (PSC)

In [58]:
INDEX_ENDPT_NAME = "index_endpoint_for_demo_1"
SHARED_VPC_PROJECT_ID = "shared-vpc-admin"
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"
ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [59]:
from google.cloud.aiplatform_v1 import IndexEndpointServiceClient
import time

In [60]:
index_endpoint_request = {"display_name": INDEX_ENDPT_NAME}
index_endpoint_request["private_service_connect_config"] = {"enable_private_service_connect": True,
                                                                "project_allowlist": [PROJECT_ID, SHARED_VPC_PROJECT_ID]}

index_endpoint_client = IndexEndpointServiceClient(client_options=dict(api_endpoint=ENDPOINT))

r = index_endpoint_client.create_index_endpoint(
    parent=PARENT,
    index_endpoint=index_endpoint_request)

In [62]:
while True:
    if r.done():
        break
    time.sleep(5)
    print('.', end='')

In [63]:
index_endpoint = r.result()

In [64]:
INDEX_ENDPOINT_NAME = index_endpoint.name
INDEX_ENDPOINT_NAME

'projects/896267025569/locations/us-central1/indexEndpoints/4699638152556445696'

In [12]:
INDEX_ENDPOINT_NAME = "4699638152556445696"

In [13]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(INDEX_ENDPOINT_NAME)

## Deploy Indexes

### Deploy ANN Index

In [14]:
DEPLOYED_INDEX_ID = "stream_tree_ah_glove_deployed_unique"

In [67]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, 
    deployed_index_id=DEPLOYED_INDEX_ID,
    min_replica_count=2,
    max_replica_count=2,
    machine_type="e2-standard-2"
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/896267025569/locations/us-central1/indexEndpoints/4699638152556445696
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/896267025569/locations/us-central1/indexEndpoints/4699638152556445696/operations/4228630117364006912
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/896267025569/locations/us-central1/indexEndpoints/4699638152556445696


[id: "stream_tree_ah_glove_deployed_unique"
index: "projects/896267025569/locations/us-central1/indexes/3053572488752529408"
create_time {
  seconds: 1692327765
  nanos: 863918000
}
private_endpoints {
  service_attachment: "projects/n10ed740331319020-tp/regions/us-central1/serviceAttachments/sa-gkedpm-f1608166dec39d7b39e0683f85fe14"
}
index_sync_time {
  seconds: 1692328736
  nanos: 998601000
}
dedicated_resources {
  machine_spec {
    machine_type: "e2-standard-2"
  }
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

### Deploy Brute Force Index

In [34]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "glove_brute_force_deployed_unique"

In [36]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=brute_force_index, 
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    min_replica_count=2,
    max_replica_count=2,
    # machine_type=""
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552/operations/879733203437355008
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552


[id: "glove_brute_force_deployed_unique"
index: "projects/1023019892523/locations/us-central1/indexes/4679758982326255616"
create_time {
  seconds: 1691031373
  nanos: 588240000
}
private_endpoints {
  service_attachment: "projects/q11906587d2a7dda8-tp/regions/us-central1/serviceAttachments/sa-gkedpm-5f0baff4586951a9ca3e7156d753f5"
}
index_sync_time {
  seconds: 1691031708
  nanos: 737742000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
, id: "tree_ah_glove_deployed_unique"
index: "projects/1023019892523/locations/us-central1/indexes/2049656799941885952"
create_time {
  seconds: 1691031295
  nanos: 81854000
}
private_endpoints {
  service_attachment: "projects/q11906587d2a7dda8-tp/regions/us-central1/serviceAttachments/sa-gkedpm-5f0baff4586951a9ca3e7156d753f5"
}
index_sync_time {
  seconds: 1691031510
  nanos: 319646000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

## Create Config Needed for PSC

Ask Walmart Strati team to create these for you

In [ ]:
SERVICE_ATTACHMENT_URI="projects/q11906587d2a7dda8-tp/regions/us-central1/serviceAttachments/sa-gkedpm-5f0baff4586951a9ca3e7156d753f5"

NETWORK_NAME="projects/860472322816/global/networks/host-shared-vpc"
ADDRESS_NAME="vme-psc-address-svc-for-demo"
REGION="us-central1"
SUBNET_NAME="projects/axel-host-shared-vpc/regions/us-central1/subnetworks/host-shared-vpc"
SVC_PROJECT="axel-argolis-1"
ENDPOINT_NAME="psc-endpoint-for-vme-demo"

gcloud compute addresses create ${ADDRESS_NAME:?} \
    --region=${REGION:?} \
    --subnet=${SUBNET_NAME:?} \
    --project=${SVC_PROJECT:?}

gcloud compute forwarding-rules create ${ENDPOINT_NAME:?} \
    --network=${NETWORK_NAME:?} \
    --address=${ADDRESS_NAME:?} \
    --target-service-attachment=${SERVICE_ATTACHMENT_URI:?} \
    --project=${SVC_PROJECT:?} \
    --region=${REGION:?}

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).

The `filter` parameter is an optional way to filter for a subset of embeddings. In this case, only embeddings that have the `class` set as `even` are returned.

Filters are applied before and more info on the methods can be found here:

https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndexEndpoint#google_cloud_aiplatform_MatchingEngineIndexEndpoint_find_neighbors

```
List[Namespace]
Optional. A list of Namespaces for filtering the matching results. For example, [Namespace("color", ["red"], []), Namespace("shape", [], ["squared"])] will match datapoints that satisfy "red color" but not include datapoints with "squared shape". Please refer to https://cloud.google.com/vertex-ai/docs/matching-engine/filtering#json for more detail.
```

In [15]:
NUM_NEIGHBOURS

10

In [18]:
ENDPT_IP = "250.0.26.250"

In [19]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import MatchNeighbor, Namespace
import grpc
from google.cloud.aiplatform.matching_engine._protos import match_service_pb2
from google.cloud.aiplatform.matching_engine._protos import (
    match_service_pb2_grpc)

In [20]:
def vme_grpc_match(endpt_ip, deployed_index_id, n_matches, embeddings, filter=[]):
    # Set up channel and stub
    channel = grpc.insecure_channel("{}:10000".format(endpt_ip))
    stub = match_service_pb2_grpc.MatchServiceStub(channel)

    # Create the batch match request
    batch_request = match_service_pb2.BatchMatchRequest()
    batch_request_for_index = (
        match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
    )
    batch_request_for_index.deployed_index_id = deployed_index_id
    b_requests = []
    for query in embeddings:
        request = match_service_pb2.MatchRequest(
            num_neighbors=n_matches,
            deployed_index_id=deployed_index_id,
            float_val=query,
        )
        for namespace in filter:
            restrict = match_service_pb2.Namespace()
            restrict.name = namespace.name
            restrict.allow_tokens.extend(namespace.allow_tokens)
            restrict.deny_tokens.extend(namespace.deny_tokens)
            request.restricts.append(restrict)
        b_requests.append(request)

    batch_request_for_index.requests.extend(b_requests)
    batch_request.requests.append(batch_request_for_index)

    # Perform the request
    response = stub.BatchMatch(batch_request)

    # Wrap the results in MatchNeighbor objects and return
    neighbors = [
                    [
                        MatchNeighbor(id=neighbor.id, distance=neighbor.distance)
                        for neighbor in embedding_neighbors.neighbor
                    ]
                    for embedding_neighbors in response.responses[0].responses
                ]

    return neighbors


In [21]:
response = vme_grpc_match(ENDPT_IP, DEPLOYED_INDEX_ID, NUM_NEIGHBOURS, test[:3].tolist(), [Namespace("class", ["even"])])
response

[[MatchNeighbor(id='310080', distance=18.029903411865234),
  MatchNeighbor(id='702494', distance=16.84667205810547),
  MatchNeighbor(id='505832', distance=16.334247589111328),
  MatchNeighbor(id='825418', distance=16.22643280029297),
  MatchNeighbor(id='707954', distance=16.183256149291992),
  MatchNeighbor(id='436242', distance=16.11363410949707),
  MatchNeighbor(id='955052', distance=16.022321701049805),
  MatchNeighbor(id='20414', distance=15.94982624053955),
  MatchNeighbor(id='208920', distance=15.804502487182617),
  MatchNeighbor(id='380434', distance=15.756570816040039)],
 [MatchNeighbor(id='752950', distance=21.807270050048828),
  MatchNeighbor(id='690062', distance=19.74156951904297),
  MatchNeighbor(id='52842', distance=19.29770278930664),
  MatchNeighbor(id='484314', distance=18.903165817260742),
  MatchNeighbor(id='317160', distance=18.501405715942383),
  MatchNeighbor(id='1150802', distance=18.053863525390625),
  MatchNeighbor(id='377396', distance=18.051637649536133),
  M

In [23]:
response = vme_grpc_match(ENDPT_IP, DEPLOYED_INDEX_ID, NUM_NEIGHBOURS, test[:1].tolist(), [Namespace("class", ["odd"])])
response

[[MatchNeighbor(id='899605', distance=20.079139709472656),
  MatchNeighbor(id='792161', distance=18.574535369873047),
  MatchNeighbor(id='729301', distance=18.31948471069336),
  MatchNeighbor(id='1093903', distance=17.839481353759766),
  MatchNeighbor(id='296543', distance=16.99886703491211),
  MatchNeighbor(id='21495', distance=16.8770694732666),
  MatchNeighbor(id='689839', distance=16.852933883666992),
  MatchNeighbor(id='518781', distance=16.65009307861328),
  MatchNeighbor(id='405251', distance=16.278234481811523),
  MatchNeighbor(id='1142011', distance=16.186227798461914)]]

In [24]:
%%timeit


vme_grpc_match(ENDPT_IP, DEPLOYED_INDEX_ID, NUM_NEIGHBOURS, test[:1].tolist(), [Namespace("class", ["odd"])])

7.3 ms ± 245 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Compute Recall

Use the deployed brute force Index as the ground truth to calculate the recall of ANN Index. Note that you can run multiple queries in a single match call.

In [55]:
# Retrieve nearest neighbors for both the tree-AH index and the brute-force index
tree_ah_response_test = vme_grpc_match(ENDPT_IP, DEPLOYED_INDEX_ID, NUM_NEIGHBOURS, list(test))

brute_force_response_test = vme_grpc_match(ENDPT_IP, DEPLOYED_BRUTE_FORCE_INDEX_ID, NUM_NEIGHBOURS, list(test))

In [56]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute-force option.
recalled_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(
    tree_ah_response_test, brute_force_response_test
):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    recalled_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = recalled_neighbors / len(
    [neighbor for neighbors in brute_force_response_test for neighbor in neighbors]
)

print("Recall: {}".format(recall))

Recall: 0.5787


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [61]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552
Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552/operations/6281238006514843648
MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552
Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552
Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552/operations/3707430819472605184
MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/1023019892523/locations/us-central1/indexEndpoints/5679804390207127552
Deleting MatchingEngineIndexEndpoint : projects/1023019892523/locations/us-c

In [53]:
# Delete indexes
tree_ah_index.delete()
brute_force_index.delete()

Deleting MatchingEngineIndex : projects/896267025569/locations/us-central1/indexes/3778652028759179264
Delete MatchingEngineIndex  backing LRO: projects/896267025569/locations/us-central1/indexes/3778652028759179264/operations/3508054176984727552
MatchingEngineIndex deleted. . Resource name: projects/896267025569/locations/us-central1/indexes/3778652028759179264
